In [1]:
import torch
import random
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

from tqdm import tqdm

import torch.nn as nn
import torch.nn.functional as F

import math
import time

import skrf
from skrf import Network

import utils

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


# Grab some data

In [3]:
ROOT_PATH = "/scratch1/04703/sravula/UTAFSDataNew/new_data"
CHIP_NUM = 9

In [4]:
test_net = utils.get_network_from_file(ROOT_PATH, CHIP_NUM)['network']

In [6]:
print("S-parameters shape: ", test_net.s.shape)
print("S-parameters type: ", test_net.s.dtype)

S-parameters shape:  (1000, 4, 4)
S-parameters type:  complex128


In [7]:
N_PORTS = test_net.nports
N_FREQS = test_net.frequency.npoints

print("N_FREQS: ", N_FREQS)
print("N_PORTS: ", N_PORTS)

N_FREQS:  1000
N_PORTS:  4


In [8]:
reciprocity = skrf.network.reciprocity(test_net.s)

EPS_SMALL = 1e-6

IS_RECIPROCAL = False
if np.sum(np.abs(reciprocity)) < EPS_SMALL:
    IS_RECIPROCAL = True
    
print(IS_RECIPROCAL)

True


# Grab a toch tensor from this guy

In [9]:
x_test = utils.network_to_sparams(test_net)

NameError: name 'network_to_sparams' is not defined

In [ ]:
print(x_test.shape)
print(x_test.dtype)

In [ ]:
plt.figure()
for i in range(x_test.shape[1]//2):
    plt.plot(x_test[0, 2*i])
    plt.plot(x_test[0, 2*i+1])
plt.show()